In [26]:
import os,json
import pandas as pd
from pycocotools.coco import COCO
import urllib.request
import skimage.io as io
import pathlib
import pylab
import cv2
from skimage.io import imread, imshow
import imutils
import numpy as np
from PIL import Image
import math
import copy
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [27]:
def mask_rotated (original_json, destination, rotate_angle) :
    
    annFile= original_json
    coco=COCO(annFile)
    rotated_data_1 = {}
    images = []
    nonrotate_annos = []
    nonrotate_pts = []
    rotate_annos = []
    angle = 0
    angle1 = rotate_angle
    rotated_data_1['infos'] = coco.dataset['infos']
    imgIds = coco.getImgIds()

    destination_path_images = destination+'/Rotated_images'
    destination_path_annotations = destination+'/Annotation_json'
    destination_path_rotations = destination+'/Validation_images'
    pathlib.Path(destination_path_images).mkdir(parents=True, exist_ok=True) 
    pathlib.Path(destination_path_annotations ).mkdir(parents=True, exist_ok=True) 
    pathlib.Path(destination_path_rotations ).mkdir(parents=True, exist_ok=True) 


    for i in tqdm (range(len(imgIds))):

        img = coco.loadImgs(imgIds[i])[0]
        IMAGE = imread(img['file_name'])
        images.append(img)
        rotated_data_1['images'] = images
        catIds=[]
        annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
        anns_segment = coco.loadAnns(annIds)

        hgt = IMAGE.shape[0]
        wid = IMAGE.shape[1]

        if wid < hgt:

    #------------------- Rotate Image ------------------------------------------------------------------------

            (cX, cY) = (wid // 2, hgt // 2)

            # rotate our image by 90 degrees around the image
            M = cv2.getRotationMatrix2D((cX, cY), angle1, 1.0)
            rotated = cv2.warpAffine(IMAGE, M, (wid, hgt))
            # entire rotated image still views in the viewing area
            rotated = imutils.rotate_bound(IMAGE,angle1)
            debug_image = rotated.copy()

            (h1, w1) = IMAGE.shape[:2]
            (h, w) = rotated.shape[:2]
            (cX1, cY1) = (w1 // 2, h1 // 2)
            (cX, cY) = (w // 2, h // 2)

            for id, ann in enumerate(anns_segment):

                image= []
                r_axis = []

                if ann['image_id']== imgIds[i]:

    #------------------- Rotate Polygon ---------------------------------------------------------

                    segmnt_val = ann["segmentation"]

                    for val in segmnt_val:

                        r_axis = val   

                        for k in range(0, len(r_axis), 2):

                            angle = math.radians(angle1)
                            temp_point = r_axis[k] - cX1, r_axis[k+1] - cY1
                            temp_point = (temp_point[0]*math.cos(angle)-temp_point[1]*math.sin(angle) , temp_point[0]*math.sin(angle)+temp_point[1]*math.cos(angle))
                            temp_point = temp_point[0]+cX, temp_point[1]+cY

                            r_axis[k] =  int(temp_point[0])
                            r_axis[k+1] = int(temp_point[1])

                            pts = np.array(r_axis, np.int32)

                            pts = pts.reshape((-1, 1, 2))

                        isClosed = True
                        # Green color in BGR
                        color = (255, 0, 0)
                        thickness = 5

                        debug_image = cv2.polylines(debug_image, pts, isClosed, color, thickness)  

                        segments = []
                        segments.append(r_axis)

    #------------------- Rotate Bounding Box --------------------------------------------------------                       

                    rect = cv2.boundingRect(pts) #creating bounding box from the rotating polygon points
                    color = (0, 255,255) 
                    thickness = 3

                    start_point = (rect[0], rect[1]) 
                    end_point = (rect[0]+rect[2], rect[1]+rect[3])

                    debug_image = cv2.rectangle(debug_image, start_point, end_point, color, thickness) 

                    rotate_annos.append({"segmentation" : segments,
                            "area" : rect[2]*rect[3],
                            "bbox" : list(rect),
                            "iscrowd" : ann['iscrowd'],
                            "id": ann['id'],
                            "image_id" : ann['image_id'],
                            "category_id" : ann['category_id']})
            
            head1, tail1 = os.path.split(img['file_name'])
            image_name1 = tail1 
            
            file_name = destination_path_images+'/'+image_name1
            plt.imsave(file_name, rotated)
            

            ori_pil =Image.open(img['file_name']) # original pic
            rotate_pil =Image.open(file_name)  # rotated pic
            masked_pil = Image.fromarray(debug_image) # masked pic


            new_image = Image.new('RGB',(3*max(ori_pil.size[0], rotate_pil.size[0], masked_pil.size[0]),max(ori_pil.size[1], rotate_pil.size[1], masked_pil.size[1])), (250,250,250))
            new_image.paste(ori_pil,(0,0))
            new_image.paste(rotate_pil,(ori_pil.size[0],0))
            new_image.paste(masked_pil,(rotate_pil.size[0]+ori_pil.size[0],0))


            new_image.save(destination_path_rotations+'/'+image_name1,"JPEG")

            rotated_data_1['images'][i]['file_name'] = file_name
            rotated_data_1['images'][i]['width'] = rotated.shape[1]
            rotated_data_1['images'][i]['height'] = rotated.shape[0]
            
        else:


            for id, anno in enumerate(anns_segment):

                cat_id = []
                bbox = []

    #------------------- Create Polygon ---------------------------------------------------------

                cat_id = int(anno["category_id"])
                bbox = anno["bbox"] # [x,y,w,h] object positioning frame
                assert len(bbox) == 4
                xmin = bbox[0]
                ymin = bbox[1]
                xmax = xmin + bbox[2]
                ymax = ymin + bbox[3]

                nonrotate_pts = anno['segmentation']
                pts1 = np.array(nonrotate_pts, np.int32)
                pts1 = pts1.reshape((-1, 1, 2))

                isClosed = True
                color = (255, 0, 0)
                thickness = 5

                # Draw a Green polygon with 
                # IMAGE = cv2.polylines(IMAGE, pts1, isClosed, color, thickness) 

    #------------------- Create Bounding Box-------------------------------------------------------

                start_point = (xmin, ymin) 
                end_point = (xmax, ymax) 

                # Blue color in BGR 
                color = (0, 255,255) 
                thickness = 3

                # Draw a rectangle with blue line borders of thickness of 2 px 

               # IMAGE = cv2.rectangle(IMAGE, start_point, end_point, color, thickness) 

    #--------------------- JSON Creation ------------------------------------------------------------            

                box_val1 = list(start_point + end_point)

                rotate_annos.append({"segmentation" : nonrotate_pts,
                            "area" : bbox[2]*bbox[3],
                            "bbox" : bbox,
                            "iscrowd" : anno['iscrowd'],
                            "id": anno['id'],
                            "image_id" : imgIds[i],
                            "category_id" : anno['category_id']})
            
            head, tail = os.path.split(img['file_name'])
            image_name = tail 
            
            file_name = destination_path_images+'/'+image_name

            #file_name = destination_path_images+'/'+img['file_name'].split("/")[6]
            plt.imsave(file_name, IMAGE)

            rotated_data_1['images'][i]['file_name'] = file_name

            
    rotated_data_1['annotations'] = rotate_annos
    rotated_data_1['licenses'] = coco.dataset['licenses']
    cats = coco.loadCats(coco.getCatIds())       
    rotated_data_1['categories'] = cats
    
    # Save to a new JSON file for easy viewing of data characteristics

    json.dump(rotated_data_1,open(destination_path_annotations+'/rotated_'+str(rotate_angle)+'.json','w'))

    print('------------------------------------------------------------')

    print('CREATED')

In [28]:
original_json_path = '/data/cv_inspection_repair/training-files/tmp/annotations/annotations.json'
destination_folder = '/workspace/abhishek/image_annotation/FINAL/final_presentation/ROTATE'
angle_rotation = -60

mask_rotated (original_json_path, destination_folder, angle_rotation)

  1%|          | 1/150 [00:00<00:19,  7.71it/s]

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


100%|██████████| 150/150 [00:21<00:00,  6.89it/s]

------------------------------------------------------------
CREATED


In [25]:
!pip install tqdm

distutils: /usr/local/lib/python3.8/dist-packages
sysconfig: /usr/lib/python3.8/site-packages
distutils: /usr/local/lib/python3.8/dist-packages
sysconfig: /usr/lib/python3.8/site-packages
distutils: /usr/local/include/python3.8/UNKNOWN
sysconfig: /usr/include/python3.8
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
distutils: /usr/local/lib/python3.8/dist-packages
sysconfig: /usr/lib/python3.8/site-packages
distutils: /usr/local/lib/python3.8/dist-packages
sysconfig: /usr/lib/python3.8/site-packages
distutils: /usr/local/include/python3.8/UNKNOWN
sysconfig: /usr/include/python3.8
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
